# NEURAL NETWORKS

In [1]:
import tensorflow as tf
from sklearn.datasets import load_iris
from sklearn import svm
import pylab as pl
import numpy as np
import pandas as pd  
from sklearn.model_selection import train_test_split

#others
import requests, json
from sklearn.externals import joblib

C:\Users\erika\Anac\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
#read the data
df = pd.read_csv('./data/joinedData.csv', sep=r'\s*(?:\||\#|\,)\s*',
                 engine='python')

#change the classes to numbers
Class = {'LUAD': 0,'BRCA': 1,'KIRC': 2,'PRAD': 3,'COAD': 4} 
df.Class = [Class[item] for item in df.Class] 
df = df.drop('Unnamed: 0',1)
df = df.drop('Unnamed: 0.1',1)
df

#separate the data into X and y
X = df.drop('Class', axis=1).values
y = df['Class'].values
y = np.asarray(y)

#standardize the data
X = (X - X.mean()) / (X.max() - X.min())

#Split the data set to test and train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)  
X_train

array([[-0.31009073, -0.18797149, -0.2357943 , ...,  0.12921791,
        -0.03963684, -0.31009073],
       [-0.31009073, -0.15962538, -0.22673834, ...,  0.12077017,
        -0.06004286, -0.31009073],
       [-0.31009073, -0.1331768 , -0.14456983, ...,  0.13259054,
         0.00935231, -0.31009073],
       ...,
       [-0.31009073, -0.18420899, -0.22079257, ...,  0.18429171,
        -0.16373322, -0.2214906 ],
       [-0.31009073, -0.10891886, -0.21409106, ...,  0.13492128,
        -0.0727008 , -0.31009073],
       [-0.31009073, -0.26460414, -0.21590543, ...,  0.14518761,
         0.12361397, -0.31009073]])

In [4]:
#get the estimator
estimator = tf.estimator.DNNClassifier(
    feature_columns=[tf.feature_column.numeric_column('x', shape=X_train.shape[1:])],
    hidden_units=[1000, 500, 250], 
    optimizer=tf.train.ProximalAdagradOptimizer(
      learning_rate=0.01,
      l1_regularization_strength=0.001
    ), #optimizer was used to improve the estimator
    n_classes=5) #the number of label classes, we have 5

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\erika\\AppData\\Local\\Temp\\tmpm7ulrxoh', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000023A4B0A6160>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [5]:
# defining the training inputs
train = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    batch_size=X_test.shape[0],
    num_epochs=None,
    shuffle=False,
    num_threads=1
    ) 

In [6]:
estimator.train(input_fn = train,steps=1000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\erika\AppData\Local\Temp\tmpm7ulrxoh\model.ckpt.
INFO:tensorflow:loss = 260.84708, step = 1
INFO:tensorflow:global_step/sec: 1.86489
INFO:tensorflow:loss = 3.527764, step = 101 (53.623 sec)
INFO:tensorflow:global_step/sec: 1.90528
INFO:tensorflow:loss = 0.30487862, step = 201 (52.488 sec)
INFO:tensorflow:global_step/sec: 1.8966
INFO:tensorflow:loss = 0.13107656, step = 301 (52.737 sec)
INFO:tensorflow:global_step/sec: 1.92467
INFO:tensorflow:loss = 0.11306629, step = 401 (51.947 sec)
INFO:tensorflow:global_step/sec: 1.93759
INFO:tensorflow:loss = 0.05697407, step = 501 (51.610 sec)
INFO:tensorflow:global_step/sec: 1.94645
INFO:tensorflow:loss = 0.040927377, step = 601 (51.383 sec)
INFO:tensorflow:global_step/sec: 1.9365
INFO:tensorflow:loss = 0.047948595, step = 701 (51.630 sec)
INFO:tensorflow:global_step/sec: 1.92025
INFO:tensorflow:loss = 0.053135067, step = 801 (52.077 sec)
INFO:tenso

In [7]:
# defining the test inputs
input_fn2 = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test},
    y=y_test, 
    shuffle=False,
    batch_size=X_test.shape[0],
    num_epochs=None)

In [8]:
#evaluate the estimator
estimator.evaluate(input_fn2,steps=1000) 

INFO:tensorflow:Starting evaluation at 2019-02-28-03:20:53
INFO:tensorflow:Restoring parameters from C:\Users\erika\AppData\Local\Temp\tmpm7ulrxoh\model.ckpt-1000
INFO:tensorflow:Evaluation [100/1000]
INFO:tensorflow:Evaluation [200/1000]
INFO:tensorflow:Evaluation [300/1000]
INFO:tensorflow:Evaluation [400/1000]
INFO:tensorflow:Evaluation [500/1000]
INFO:tensorflow:Evaluation [600/1000]
INFO:tensorflow:Evaluation [700/1000]
INFO:tensorflow:Evaluation [800/1000]
INFO:tensorflow:Evaluation [900/1000]
INFO:tensorflow:Evaluation [1000/1000]
INFO:tensorflow:Finished evaluation at 2019-02-28-03:24:18
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.99378884, average_loss = 0.019068621, global_step = 1000, loss = 3.070048


{'accuracy': 0.99378884,
 'average_loss': 0.019068621,
 'global_step': 1000,
 'loss': 3.070048}

In [9]:
#predict input
pred_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test},
    shuffle=False)

In [10]:
#predict the results
pred_results = estimator.predict(input_fn=pred_input_fn)

In [11]:
#print the results, example of the result is array([b'2'], dtype=object) 
#b = 2 = KIRC
for i in pred_results:
    print (i)

INFO:tensorflow:Restoring parameters from C:\Users\erika\AppData\Local\Temp\tmpm7ulrxoh\model.ckpt-1000
{'logits': array([12.554505  , -1.4340204 ,  0.748541  ,  1.3832573 ,  0.35682046],
      dtype=float32), 'probabilities': array([9.9997258e-01, 8.4110206e-07, 7.4597256e-06, 1.4072672e-05,
       5.0419749e-06], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([ 0.68341154, 12.227181  , -4.5726275 ,  1.8584318 ,  1.3161865 ],
      dtype=float32), 'probabilities': array([9.6956901e-06, 9.9994075e-01, 5.0572002e-08, 3.1396667e-05,
       1.8255323e-05], dtype=float32), 'class_ids': array([1], dtype=int64), 'classes': array([b'1'], dtype=object)}
{'logits': array([ 1.4622979,  0.727489 , -3.5154781, 10.137078 , -3.3555121],
      dtype=float32), 'probabilities': array([1.7079693e-04, 8.1913633e-05, 1.1766822e-06, 9.9974471e-01,
       1.3808032e-06], dtype=float32), 'class_ids': array([3], dtype=int64), 'classes': array([b'

# Run Model API

## Persist Model and Data

### Python Pickle

In [15]:
import pickle

with open("python_DNN_model.pkl", "wb") as file_handler:
    pickle.dump(tf.estimator.DNNClassifier, file_handler)
    
with open("python_DNN_model.pkl", "rb") as file_handler:
    loaded_pickle = pickle.load(file_handler)
    
loaded_pickle

tensorflow.python.estimator.canned.dnn.DNNClassifier

#### scikit-learn's joblib

In [18]:
BASE_URL = "http://localhost:5000"

joblib.dump(tf.estimator.DNNClassifier, "DNN_model.pkl")

joblib.dump(X, "X.pkl")
joblib.dump(y, "Y.pkl")

['Y.pkl']

### Predict API

In [31]:
Cancer = {"gene_0":0.00001, "gene_1":0.002, "gene_2": 0.0002}

response = requests.post("{}/predict".format(BASE_URL), json = Cancer)

response.json()

JSONDecodeError: Expecting value: line 3 column 2 (char 3)